In [ ]:
#Take USGS DEM Data and turn it into a useable CM1 format. This code does not 
#  - do any smoothing beyond bilinear interpolation
#  - Isolate the terrain 
# These features can be added but will likely vary on a case-by-case basis

#Import necessary libraries
from osgeo import gdal 
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Transformer
import scipy.interpolate as interp
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as patches

#Set some user-defined variables
file_lats = [40,39,38,37,36,35,34] #Needs to be descending for this to work
file_lons = [-87,-86,-85,-84,-83,-82,-81,-80,-79,-78] #I used negatives here and just dealt with it in the code. Thinks this makes the most sense
dir_to_data = '../'
cm1_dx = 1000.0 #in meters

#Read in the USGS DEM files. Assumes all DEM files are in northern and western hemispheres
count_i = 0
for i in file_lats:
    count_j = 0
    for j in file_lons:
        file_string = ('/Users/roger/Desktop/cm1_dem_data/USGS_1_n%02dw%03d.tif' %(i,j*-1))
        #print(file_string)
        geo = gdal.Open(file_string)
        #print(geo)
        #Read heights out of the IMG file
        dem_heights_temp = geo.ReadAsArray()
        
        #Grab some info about the 
        width = geo.RasterXSize
        height = geo.RasterYSize
        gt = geo.GetGeoTransform()
        #We are given the farthest north and west points of the box, so calculate the souther lat and eastern
        # lon. Given d_lat and d_lon in the metadata in the geotransform
        #gt[0] is the western most point
        #gt[3] is the northern most point
        #gt[1] is d_lon (should be positive)
        #gt[5] is d_lat (should be negative)
        #gt[2] and gt[4] are 0 in the DEM metadata and can be ignored
        minx = gt[0]
        miny = gt[3] + width*gt[4] + height*gt[5] 
        maxx = gt[0] + width*gt[1] + height*gt[2]
        maxy = gt[3] 
        
        lon_tmp = np.arange(minx,maxx,gt[1])
        lat_tmp = np.arange(miny,maxy,gt[5]*-1)
        if np.size(lon_tmp) > width:
            #Due to rounding errors, the lon and lat arrays can sometimes change size from
            # the original DEM files. Just trim them to match the shape of the OG file.
            size_diff = np.size(lon_tmp) - width
            trim = -1*(size_diff)
            lon_tmp = lon_tmp[0:trim]
        if count_j == 0:
            dem_heights_lons = dem_heights_temp
            lons = lon_tmp 
        if count_j > 0:
            #Append the current terrain field to the old one
            dem_heights_lons = np.append(dem_heights_lons,dem_heights_temp,axis=1)
            lons = np.append(lons,lon_tmp)
        count_j += 1
    if np.size(lat_tmp) > height:
        #Due to rounding errors, the lon and lat arrays can sometimes change size from
        # the original DEM files. Just trim them to match the shape of the OG file.
        size_diff = np.size(lat_tmp) - height
        trim = -1*(size_diff + 1)
        lat_tmp = lat_tmp[0:trim]
    if count_i == 0:
        dem_heights_lats = dem_heights_lons
        lats = lat_tmp
    if count_i > 0:
        #Again, append the current terrain field to the old one
        old_lats = lats
        dem_heights_lats = np.append(dem_heights_lats,dem_heights_lons,axis=0)
        old_lats = lats
        lats = np.append(lat_tmp,old_lats)
    count_i += 1
#Flip because of how it was appended (going down instead of going up like how the array reads them in)
terrain_height_dem = dem_heights_lats
lats_dem = lats
lons_dem = lons

#Now do a transform from lat/lon to UTM. This produces x and y coordinates in meters. Zone 14 is for 
# central OK where data was tested. 17 more appropriate for southern Appalachians
trans = Transformer.from_crs("epsg:4326","proj=utm +zone=17 +ellps=WGS84",always_xy=True,)
#Lons/lats need to be same size, so let's force that to make it happen?

lons_transf,lats_transf = np.meshgrid(lons_dem,lats_dem)
xx,yy = trans.transform(lons_transf,lats_transf)


#Set UTM coordinates with 0,0 at the lower left corner of the domain
xx_0 = xx[0,:]-xx[0,0]
yy_0 = yy[:,0]-yy[0,0]



#Set up a grid with CM1 dx/dy. This will set up the interpolation to these values
interp_x_vals = np.arange(0,xx_0[-1],cm1_dx)
interp_y_vals = np.arange(0,yy_0[-1],cm1_dx)

#Perform a bilinear interpolation using the scipy interp2d function
XX,YY = np.meshgrid(xx_0,yy_0)

f = interp.interp2d(xx_0,yy_0,terrain_height_dem[0:-5])
#perform the actual interpolation at the CM1 coordinates
interp_zs = f(interp_x_vals,interp_y_vals)

In [ ]:
#Import the Gaussian filter from ndimage
from scipy.ndimage import gaussian_filter
#Create meshgrids for plotting for both the interp vals and the original grid
X,Y = np.meshgrid(interp_x_vals,interp_y_vals)
XX,YY = np.meshgrid(xx_0,yy_0)
#Rearrange the array so it's right for CM1
zs_tmp = np.flipud(interp_zs)
#This gets the "standard deviation" for the gaussian filter to filter out waves 6*dx and lower
#This is taken from a MATLAB program I downloaded called FILT2D which is a geospatial filter
sigma_guess = 6. /(2 * np.pi)
#Perform the low pass filtering
zs_filt = gaussian_filter(zs_tmp,sigma=(sigma_guess,sigma_guess))
#Lower the terrain height by 300 m
zs_lower = zs_filt - 300.
#If the terrain is lower than 0, set it to 0
zs_lower = np.where(zs_lower < 0, 0, zs_lower)
# This is where we define the parameters of the ellipse. The values here are in meters.
# I just eyeballed the values until they looked correct and did some trial and error.
# If you want to change these, just make some plots with the ellipse plotted on top and you can
# sort of adjust it as needed
g_ell_center = (425000, 320000) #Center of elipse
g_ell_width = 250000 # Width of ellipse
g_ell_height = 125000 # height of ellipse
angle = 35. # Angle of ellipse
cos_angle = np.cos(np.radians(180.-angle))
sin_angle = np.sin(np.radians(180.-angle))

xc = X - g_ell_center[0] #Center coordinates on the ellipse
yc = Y - g_ell_center[1] 

xct = xc * cos_angle - yc * sin_angle
yct = xc * sin_angle + yc * cos_angle 

#Calculates the radius of the ellipse from the center points
rad_cc = (xct**2/(g_ell_width/2.)**2) + (yct**2/(g_ell_height/2.)**2)

#Plots to show the ellipse on top of the terrain

#fig4,ax4 = plt.subplots(figsize=(16,16),subplot_kw={"projection": ccrs.PlateCarree()})
fig4,ax4 = plt.subplots(figsize=(16,12))
cax4 = ax4.contourf(X,Y,zs_tmp,levels = np.arange(0,1225,25), cmap = plt.get_cmap('copper'))
#I think this is plots the region of the terrain we are pulling for the final CM1 file
rect = patches.Rectangle((50000,125000),600000,400000,fill=False, edgecolor='white', linewidth=4)
#Plots the ellipse
g_ellipse = patches.Ellipse(g_ell_center, g_ell_width, g_ell_height, angle=angle, fill=False, edgecolor='orange', linewidth=4)
ax4.add_patch(rect)
ax4.add_patch(g_ellipse)
ax4.set_aspect('equal')
fig4.colorbar(cax4)
plt.show()

# Keep the terrain within the calculated ellipse (where radius <= 1)
terr_ellipse = np.where(rad_cc <= 1.,zs_lower,0)
#Between 1 and 3 ellipse radii, linearly smooth the terrain down to 0
terr_ellipse = np.where((rad_cc >=1) & (rad_cc <=3),zs_lower*((3-rad_cc)/((3-1))),terr_ellipse)

#Plot to show the terrain we are keeping for the CM1 terrain file
fig3,ax3 = plt.subplots(figsize=(16,12))
cax3 = plt.contourf(X,Y,terr_ellipse,levels=np.arange(0,1225,25), cmap = plt.get_cmap('copper') )
rect = patches.Rectangle((50000,125000),600000,400000,fill=False, edgecolor='white', linewidth=4)
ax3.add_patch(rect)
ax3.set_aspect('equal')
g_ellipse = patches.Ellipse(g_ell_center, g_ell_width, g_ell_height, angle=angle, fill=False, edgecolor='orange', linewidth=4)
ax3.add_patch(g_ellipse)
#ax3.contour(X,Y,rad_cc,levels=[1,2],colors='k')
fig3.colorbar(cax3)

In [ ]:
#This defines the bounding box (put in meters from the original large domain)
xbox_0 = 25000
xbox_1 = 625000
ybox_0 = 125000
ybox_1 = 625000
#Grab the coordinates of the bounding box
x_start = np.where(interp_x_vals==xbox_0)[0][0]
x_end = np.where(interp_x_vals==xbox_1)[0][0]
y_start = np.where(interp_y_vals==ybox_0)[0][0]
y_end = np.where(interp_y_vals==ybox_1)[0][0]
zs_save = terr_ellipse[y_start:y_end,x_start:x_end]
#Get the shape
zs_shape = np.shape(zs_save)
#Put the zs field into a writeout variable (Can't remember why I did this, but it's here. Might be unnecessary)
zs_writeout = zs_save
#Plot the field in a lazy way to make sure it looks how we want it to
plt.contourf(zs_writeout, cmap = plt.get_cmap('copper') )

#Open the binary file we'll be writing to
file = open('perts_Riggin_1000m_res.dat','wb')
#Convert to float 32 because that's what CM1 wants (discovered by trial and error and Stack Overflow)
terrain_final = np.float32(zs_writeout)
#Get shape of this field
terr_shape = np.shape(terrain_final)
#Write the terrain field line by line to the file (this is fast)
for i in range(terr_shape[0]):
    lineArr = terrain_final[i,:]
    lineArr.tofile(file)
#Close the file 
file.close()